## Matching services between Evasion and Anexo_4

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import datetime as dt
import time

In [3]:
import EvasionBuilder

In [4]:
[first_q, second_q, third_q] = EvasionBuilder.loadSinglesEvasion()

In [5]:
from Utils import TransantiagoConstants
DTPMDir = TransantiagoConstants.DTPMDir
first_semester_stops_path = os.path.join(DTPMDir,'02_Anexo_4/2017-05-06_consolidado_anexo4_(Circunvalación)_v2.xlsm')

In [6]:
first_semester_stops = pd.read_excel(first_semester_stops_path)

In [7]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [8]:
first_semester_stops_1 = first_semester_stops.loc[first_semester_stops['Orden']==1,['Orden', 'Servicio TS', 'Servicio Usuario', 'Sentido Servicio', 'Varian-te', 'Código paradero TS', 'Eje', 'Nombre Paradero']]

* Analyzing first quarter

In [9]:
first_q.head()

,FECHA,SERVICIO,PLACA PATENTE,NUMERO DE PUERTAS,PUERTA NUMERO,LUGAR INICIO,HORA INICIO,HORA,MINUTOS,INGRESAN,NO VALIDAN,TP
0,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,10,15,1,P
1,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,12,9,0,P
2,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,13,6,2,P
3,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,18,8,1,P
4,2017-01-10,214,BJFH98,3,1,AV. ADOLFO LOPEZ MATEO,07:10:00,7,20,5,1,P


In [10]:
first_semester_stops_1.head()

,Orden,Servicio TS,Servicio Usuario,Sentido Servicio,Varian-te,Código paradero TS,Eje,Nombre Paradero
0,1,101,101,I,NaN,T-4-25-OP-5,VENEZUELA,Venezuela esq. / Bolivia
77,1,101,101,R,NaN,L-12-24-15-SN,AV. LAS TORRES,Avenida Las Torres esq. / Acapulco
148,1,102,102,I,NaN,L-7-48-5-PO,GRAL. AMENGUAL,General Amengual esq. / Padre V. Irarrázaval
215,1,102,102,R,NaN,T-34-269-SN-5,AV. CAMILO HENRÍQUEZ,Parada 2 / Hospital El Peral
281,1,103,103,I,NaN,T-14-110-PO-10,AV. NUEVA PROVIDENCIA,Parada 3 / (M) Manuel Montt


In [15]:
tic = time.clock()
first_q['SENTIDO']=""

empty = 0
length_one = 0
length_two = 0
length_more_than_two = 0

for index,row in first_q.iterrows():
    evasion_servicio = row['SERVICIO']
    evasion_inicio = row['LUGAR INICIO']
    paradas_ejes = first_semester_stops_1.loc[first_semester_stops_1['Servicio Usuario']==evasion_servicio,'Eje']
    paradas_sentidos = first_semester_stops_1.loc[first_semester_stops_1['Servicio Usuario']==evasion_servicio,'Sentido Servicio']
    paradas_variantes = first_semester_stops_1.loc[first_semester_stops_1['Servicio Usuario']==evasion_servicio,'Varian-te']
    #Check for similarity between evasion_inicio and all paradas_ejes, get the max
    
    if(paradas_ejes.empty==True):
        first_q.loc[index,'SENTIDO']='empty'
        empty = empty + 1
    else:
        if(len(paradas_ejes.index)==1):
            #There is no necessity to compute similarities
            length_one = length_one + 1
            first_q.loc[index,'SENTIDO']=paradas_sentidos.values[0]
            
        elif(len(paradas_ejes.index)==2):
            length_two = length_two + 1
            #Similarities are computed. TODO: check for same similarities.
            for i in range(0,len(paradas_ejes.index)):
                similarities = {}
                similarities[i] = similar(evasion_inicio, paradas_ejes.iloc[i])
                
            max_similarity = max(similarities.values())
            sentido_index = [k for k, v in similarities.items() if v == max_similarity]
            sentido_final = paradas_sentidos.iloc[sentido_index]
            
            first_q.loc[index,'SENTIDO']=sentido_final.values[0]
        
        else:
            #There is no necessity to compute similarities
            length_more_than_two = length_more_than_two + 1
            first_q.loc[index,'SENTIDO']='more_than_two'

toc = time.clock()
print(toc-tic)

108.97388375405626
